In [1]:
key = ''

In [2]:
import os
import openai
from openai import OpenAI

In [3]:
openai.api_key=key
client=OpenAI(api_key=key)

In [5]:
import time
import random
import re

def get_cvss_metrics(description):
    prompt = f"""
    Analyze the following CVE description and calculate the CVSS v3.1 Base Score. Determine the values for each base metric: AV, AC, PR, UI, S, C, I, and A. Summarize each metric's value and provide the final CVSS v3.1 vector string.
    
    CVE Description: {description}
    
    Valid options for each metric are as follows:
    - Attack Vector (AV): Network (N), Adjacent (A), Local (L), Physical (P)
    - Attack Complexity (AC): Low (L), High (H)
    - Privileges Required (PR): None (N), Low (L), High (H)
    - User Interaction (UI): None (N), Required (R)
    - Scope (S): Unchanged (U), Changed (C)
    - Confidentiality (C): None (N), Low (L), High (H)
    - Integrity (I): None (N), Low (L), High (H)
    - Availability (A): None (N), Low (L), High (H)
    
    Ensure the final line of your response contains only the CVSS v3 Vector String in the following format:
    Example format: CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H
    """
    
    max_retries = 5  # Set a max retry limit for rate limit errors
    attempt = 0

    while attempt < max_retries:
        try:
            response = client.chat.completions.create(
                model="gpt-4",  # Use GPT-4 model here
                messages=[
                    {"role": "system", "content": "You are an expert in cybersecurity and CVE mappings."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=300
            )

            if response.choices and hasattr(response.choices[0].message, 'content'):
                content = response.choices[0].message.content.strip()
                print(f"Raw Output: {content}")  # Debug print to see the entire output

                result = content
                
            else:
                print("Error: Invalid response from OpenAI API")
                return None

        except client.RateLimitError as e:
            print(f"Rate limit error: {e}")
            time.sleep(2 ** attempt)  # Exponential backoff
            attempt += 1  # Increment retry attempt

        except Exception as e:
            print(f"Error: {e}")
            return None

    print("Error: Exceeded maximum retry attempts for rate limit.")
    return None


In [14]:
import pandas as pd 


df=pd.read_csv('cti-vsp.tsv',sep='\t').sample(n=100)
df.head()

,URL,Description,Prompt,GT
515,https://nvd.nist.gov/vuln/detail/CVE-2024-22916,"In D-LINK Go-RT-AC750 v101b03, the sprintf fun...",Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H
852,https://nvd.nist.gov/vuln/detail/CVE-2024-24328,TOTOLINK A3300R V17.0.0cu.557_B20221024 was di...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H
2,https://nvd.nist.gov/vuln/detail/CVE-2024-22137,Improper Neutralization of Input During Web Pa...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:L/UI:R/S:C/C:L/I:L/A:N
542,https://nvd.nist.gov/vuln/detail/CVE-2023-52426,libexpat through 2.5.0 allows recursive XML En...,Analyze the following CVE description and calc...,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:N/I:N/A:H
194,https://nvd.nist.gov/vuln/detail/CVE-2024-23032,Cross Site Scripting vulnerability in num para...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:C/C:L/I:L/A:N


In [15]:
# List to store CVSS results
cvss_results = []

# Iterate over the DataFrame and get CVSS metrics from GPT-3
for index, row in df.iterrows():
    description = row["Description"]
    metrics = get_cvss_metrics(description)
    cvss_results.append(metrics)

# Add CVSS results to the DataFrame
df['CVSS'] = cvss_results

# Save the results to a new TSV file
df.to_csv('output_with_cvssGPT4.tsv', sep='\t', index=False)

# Display the DataFrame with CVSS results
print(df[['URL', 'Description', 'CVSS']])

Raw Output: The CVE description does not provide quite enough information to fully determine the metrics for a CVSS v3.1 score. However, using some general assumptions based on the limited information given, I can provide an example analysis.

1. Attack Vector (AV): Since the function susceptible to stack overflow is within the "cgibin", this typically handles web (HTTP) requests, which implies a Network (N) vector.

2. Attack Complexity (AC): There's not enough data to make a strong determination here. However, stack overflow exploits are generally not straightforward and often require custom crafting, making the complexity High (H).

3. Privileges Required (PR): Since the vulnerable area is the cgibin, this typically doesn't require any special privileges to access. Therefore, None (N).

4. User Interaction (UI): Not enough information is given, but to be conservative, let's suggest Required (R), as often these types of attacks require some form of phishing to get the user to the vul

AttributeError: 'OpenAI' object has no attribute 'RateLimitError'

In [8]:
df=pd.read_csv('output_with_cvssGPT4.tsv', sep='\t')
df.head()

,URL,Description,Prompt,GT,CVSS
0,https://nvd.nist.gov/vuln/detail/CVE-2023-31001,IBM Security Access Manager Container (IBM Sec...,Analyze the following CVE description and calc...,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:N/A:N,NaN
1,https://nvd.nist.gov/vuln/detail/CVE-2024-23514,Improper Neutralization of Input During Web Pa...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:L/UI:R/S:C/C:L/I:L/A:N,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:L/I:L/A:N
2,https://nvd.nist.gov/vuln/detail/CVE-2023-32329,IBM Security Access Manager Container (IBM Sec...,Analyze the following CVE description and calc...,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:N/I:H/A:N,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:L/I:N/A:N
3,https://nvd.nist.gov/vuln/detail/CVE-2024-22226,"Dell Unity, versions prior to 5.4, contain a p...",Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:N/I:H/A:N,NaN
4,https://nvd.nist.gov/vuln/detail/CVE-2024-25312,Code-projects Simple School Managment System 1...,Analyze the following CVE description and calc...,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H


In [9]:
from cvss import CVSS3

def get_cvss_score(cvss_vector):
    c = CVSS3(cvss_vector)
    cvss_score = c.scores()[0]
    return cvss_score

In [12]:
def compute_vsp_mad(fname, col):
    cvss_prefix = 'CVSS:3.0/'   # should be empty string if the model responds with the prefix
    df = pd.read_csv(fname, sep='\t')
    error = 0
    total = 0
    for idx, row in df.iterrows():
        pred = str(row[col]).upper() if pd.notna(row[col]) else None  # Convert to string and check for NaN
        gt = str(row['GT']).upper() if pd.notna(row['GT']) else None  # Same for GT column
        try:
            if pred == gt:
                total += 1
        except Exception as e:
            print(f"Error processing row {idx}: {e}")

        try:
            pred_vector = cvss_prefix + pred
            pred_score = get_cvss_score(pred_vector)
            gt_score = get_cvss_score(gt)
            error += abs(pred_score-gt_score)
        except Exception as e:
            print('Invalid response at row {}'.format(idx+1))
            print(e)
            continue
        total += 1
            
    return error/total

In [13]:
print('Mean Absolute Deviation:', compute_vsp_mad('output_with_cvssGPT4.tsv','CVSS'))

Invalid response at row 1
can only concatenate str (not "NoneType") to str
Invalid response at row 2
Unknown metric "CVSS" in field "CVSS:3.1"
Invalid response at row 3
Unknown metric "CVSS" in field "CVSS:3.1"
Invalid response at row 4
can only concatenate str (not "NoneType") to str
Invalid response at row 5
Unknown metric "CVSS" in field "CVSS:3.1"
Invalid response at row 6
can only concatenate str (not "NoneType") to str
Invalid response at row 7
can only concatenate str (not "NoneType") to str
Invalid response at row 8
Unknown metric "CVSS" in field "CVSS:3.1"
Invalid response at row 9
Unknown metric "CVSS" in field "CVSS:3.1"
Invalid response at row 10
can only concatenate str (not "NoneType") to str
Invalid response at row 11
Unknown metric "CVSS" in field "CVSS:3.1"
Invalid response at row 12
can only concatenate str (not "NoneType") to str
Invalid response at row 13
can only concatenate str (not "NoneType") to str
Invalid response at row 14
can only concatenate str (not "NoneT